Modelo SVM

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib

# Cargar los datos desde el archivo CSV
data = pd.read_csv('appliance_usage_data2.csv')

# Convertir la columna 'Fecha' a formato datetime
data['Fecha'] = pd.to_datetime(data['Fecha'])

# Convertir la columna datetime a números
data['Fecha'] = pd.to_numeric(data['Fecha'])

# Dividir los datos en características (X) y etiquetas (y)
X = data[['Fecha', 'Hora']].values
y = data['Reporte de fallo'].values

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características (normalización)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Crear y entrenar el modelo SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = svm_model.predict(X_test)
print("Predicción: ", y_pred)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Guardar el modelo en un archivo
joblib.dump(svm_model, 'svm_model.pkl')

Regresión logística

In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar los datos
data = pd.read_csv("appliance_usage_data2.csv")

# Convertir las columnas `Fecha` y `Hora` a tipo datetime
data["Fecha"] = pd.to_datetime(data["Fecha"])
data["Hora"] = pd.to_datetime(data["Hora"])

# Extraer características numéricas
data["Dia de la semana"] = data["Fecha"].apply(lambda date: date.weekday())
data["Hora del día"] = data["Hora"].apply(lambda time: time.hour)

# Separar las variables de entrada y salida
X = data[["Dia de la semana", "Hora del día", "Watts usados"]]
y = data["Reporte de fallo"]

# Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Entrenar el modelo
model = LogisticRegression()
model.fit(X_train, y_train)

# Predecir los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Accuracy: 0.50


CNN

In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Carga tus datos desde un archivo CSV (asegúrate de que tu archivo de datos esté en el mismo directorio que este script)
data = pd.read_csv('appliance_usage_data21.csv')

# Preprocesamiento de datos
X = data[['Hora', 'Watts usados', 'Uso intensivo']].values
y = data['Reporte de fallo'].values

# Normaliza los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# División de datos en conjuntos de entrenamiento, validación y prueba
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Diseño del modelo
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(3,)),  # 3 características de entrada (Hora, Watts usados, Uso intensivo)
    layers.Dense(1, activation='sigmoid')  # Capa de salida para la clasificación binaria
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento del modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluación del modelo en el conjunto de prueba
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Pérdida en conjunto de prueba: {loss}')
print(f'Precisión en conjunto de prueba: {accuracy}')

# Hacer predicciones en nuevos datos
X_new_data = np.array([[6, 0.0, 1], [18, 30.0, 0]])
X_new_data = scaler.transform(X_new_data)  # Asegúrate de normalizar también los datos de predicción
predictions = model.predict(X_new_data)

# Las predicciones son probabilidades de fallo, puedes establecer un umbral para clasificarlas en 0 o 1 según tu criterio
threshold = 0.5
predicted_labels = (predictions > threshold).astype(int)

print("Predicciones:")
for i in range(len(X_new_data)):
    print(f'Datos: {X_new_data[i]}, Probabilidad de fallo: {predictions[i]}, Clasificación: {predicted_labels[i]}')

Epoch 1/10
21385/65625 [========>.....................] - ETA: 1:16 - loss: 0.6936 - accuracy: 0.4999

KeyboardInterrupt: 

XGBoost

In [11]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Cargar los datos
data = pd.read_csv("appliance_usage_data2.csv")

# Convertir las columnas `Fecha` y `Hora` a tipo datetime
data["Fecha"] = pd.to_datetime(data["Fecha"])
data["Hora"] = pd.to_datetime(data["Hora"])

# Extraer características numéricas
data["Dia de la semana"] = data["Fecha"].apply(lambda date: date.weekday())
data["Hora del día"] = data["Hora"].apply(lambda time: time.hour)

# Convirtiendo la columna de Watts usados a números
data["Watts usados"] = pd.to_numeric(data["Watts usados"])

# Separar las variables de entrada y salida
X = data[["Dia de la semana", "Hora del día", "Watts usados"]]
y = data["Reporte de fallo"]

# Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Crear el modelo de Gradient Boosting (XGBoost)
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Imprimir un informe de clasificación
print(classification_report(y_test, y_pred))

Accuracy: 0.50
              precision    recall  f1-score   support

           0       0.50      0.38      0.43     62807
           1       0.50      0.62      0.55     62193

    accuracy                           0.50    125000
   macro avg       0.50      0.50      0.49    125000
weighted avg       0.50      0.50      0.49    125000



In [15]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Crear el DataFrame con tus datos
data = pd.read_csv("appliance_usage_data21.csv")

# Separar las variables de entrada y salida
X = data[["Hora","Watts usados","Temperatura", "Uso intensivo"]]
y = data["Reporte de fallo"]

# Dividir los datos en un conjunto de entrenamiento y un conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de Gradient Boosting (XGBoost)
model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

# Entrenar el modelo
model.fit(X_train, y_train)

# Predecir los datos de prueba
y_pred = model.predict(X_test)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Imprimir un informe de clasificación
print(classification_report(y_test, y_pred))

Accuracy: 0.50


In [13]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Cargar y preprocesar los datos
data = pd.read_csv('appliance_usage_data21.csv')
data = data.drop(['Probabilidad de fallo'], axis=1)
X = data[['Fecha', 'Hora', 'Watts usados', 'Temperatura', 'Uso intensivo']]
y = data['Reporte de fallo']

# Normalizar las características (puedes utilizar StandardScaler de scikit-learn)
# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Precisión en el conjunto de prueba: {test_accuracy}')

# Realizar predicciones en nuevos datos
new_data = pd.DataFrame({'Fecha': ['2023-10-28 12:00:00'], 'Hora': [12], 'Watts usados': [250.0], 'Temperatura': [30.0], 'Uso intensivo': [1]})
prediction = model.predict(new_data)
print(f'Predicción de fallo: {prediction}')

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).